## Load pacakges 

In [1]:
import numpy as np, pandas as pd, geopandas as gpd, rasterio
import os
from xgboost import XGBClassifier, XGBRegressor

## Define functions 

In [11]:
def bootstrapping_train(dt_original, n_samples,  random_seed=7):
    """
    generate a new dataset using bootstrapping sampling, resample with replacement
    Args:
        dt_original: panda DataFrame containing the original data,
        n_samples: integer to indicate sample size for the new dataset
        random_seed: integer, setup random seed for reproducibility 
        

    Returns:
        A 2D NumPy array containing the new dataset
    """
    np.random.seed(random_seed)
    bootstrap_indices = np.random.choice(dt_original.index, size=n_samples, replace=True)
    bootstrapped_data = dt_original.loc[bootstrap_indices]
    return bootstrapped_data
    

def simulate_correlated_errors_vectorized(df_std_devs, corr_matrix):
    """
    Simulates correlated random errors for a DataFrame for a single simulation, using vectorization and a shared correlation matrix, with optimized scaling.

    Args:
        df_data: DataFrame containing the original data.
        df_std_devs: DataFrame containing the standard deviations for each observation and variable.
        corr_matrix: A NumPy array representing the shared correlation matrix.

    Returns:
        A 2D NumPy array containing the simulated errors for each observation and variable.
    """

    num_obs, num_vars = df_std_devs.shape

    # Generate uncorrelated standard normal random variables for all variables
    uncorrelated_errors = np.random.randn(num_obs, num_vars)

    # Cholesky decomposition of the shared correlation matrix
    L = np.linalg.cholesky(corr_matrix)

    # Correlate the uncorrelated errors for all observations
    correlated_errors = np.dot(uncorrelated_errors, L.T)

    # Scale the correlated errors using broadcasting
    simulated_errors = correlated_errors * df_std_devs.values

    return simulated_errors
    
def uncert_project_model(raster_np,  ccdc_rmse_np, bands_corr_matrix):
    """
    This code adds uncertainty caused by ccdc to each spectral band for model projection/prediction
    
    Args:
        raster_np: a 3-d numpy array, e.g. output from rasterio.read(),  first dimension indicates each band, 
        ccdc_rmse_np, a 3-d numpy array to indicate the rmse of each ccdc band, first dimension is for each band, the same dimension as raster_np
        bands_corr_matrix, correlation matrix among ccdc bands
    
    Returns:
       a numpy array with same shape as raster_np with simulated uncertainty added
    """

    ## get raster dimension
    bands, height, width = raster_np.shape[0], raster_np.shape[1], raster_np.shape[2] 
    ## reshape and convert numpy array to panda dataframe
    raster_np_reshape = pd.DataFrame((raster_np.reshape(6, -1)).T)
    rmse_np_reshape = pd.DataFrame((ccdc_rmse_np.reshape(6, -1)).T)

    ## simulate ccdc uncertainty using ccdc RMSE and correlation matrix among bands
    ccdc_error_df =  simulate_correlated_errors_vectorized(rmse_np_reshape,  bands_corr_matrix)

    ## add the simulated error to reshaped raster_np 
    raster_np_reshape_with_error = raster_np_reshape + ccdc_error_df
    raster_np_reshape_with_error = raster_np_reshape_with_error.to_numpy()
    raster_np_reshape_with_error.shape

    ## reshape raster_np to orginal dimension
    raster_np_with_uncert = raster_np_reshape_with_error.T.reshape(bands, height, width)
    raster_np_with_uncert.shape
    return raster_np_with_uncert
   
    
    
def uncert_train_model(dta, n, n_samples, ccdc_rmse, bands_corr_matrix, y_variable_nm = 'AGB', allometry_std_dev=15,  allometry_percentage=True, model_out_path = ''):

    """
    This code does the following tasks:
    1) Simulates 3 types of uncertaties: sampling uncertainty using bootstrapping, allometry uncertainty, and CCDC/COLD uncertainty 
    2) train a XGBM regression with uncertainty-added data, 
    3) and save the model
    Args:
        dta: DataFrame containing the original data,
        n: integer to indicate which interation this run is, 0 means first itervation, n will be used in the name of the output model,
        ccdc_rmse: DataFrame with the same number of rows as dta to indicate pixel-wise RMSE for each CCDC/COLD band,
        bands_corr_matrix: DataFrame for correlation matrix among CCDC/COLD bands for multi-variate error simulation, 
        y_variable_nm: string to indicate column name for y variable,
        allometry_std_dev: numeric, standard deviation or RMSE of the AGB allometry, this could be an abosulte number or percentage, see parameter below,
        allometry_percentage: True or False, True means the allometry_std_dev is percentage relative to the AGB value of each data point (this should be better),
        model_out_path: string to indciate path where you want to save the trained model

    Returns:
       no return, but save the trained model in given location
    """
   
    ## load training data using bootstrapping mehtod, bootsrapping_train is a self-defined function 
    random_state = n
    dt_train = bootstrapping_train(dta, n_samples, random_seed=random_state)

    ######### Uncertainty in samples ###########
    ## Sample allometry uncertainty in target variable (i.e. y variable) using Monte Carlo to simulate normal-distributed random error
    np.random.seed(random_state)
    if not allometry_percentage:
        dt_train['y_error'] = np.random.normal(loc=0, scale=allometry_std_dev , size = dt_train.shape[0])
    else:
        dt_train['y_error'] = np.random.normal(loc=0, scale=dt_train[y_variable_nm]*allometry_std_dev/100 , size = dt_train.shape[0])
    dt_train['y2'] = dt_train['y_error'] + dt_train[y_variable_nm]
    
    ## Sample uncertainty caused by CCDC, need to have pixel-wise RMSE for each band, and correlation matrix among bands to use multi-variate normal distribution for error simulation 
    ## ccdc_bands_std_devs, a panda dataframe with same # of observation as dta; this is the RMSE for each CCDC band
    ## corr_matrix,  a panda dataframe for correlation matrix among all CCDC bands; the sequence of band in these 2 dataframe should be the same

    ccdc_error_df =  simulate_correlated_errors_vectorized(ccdc_rmse,  bands_corr_matrix)

    ## add the simulated ccdc_error to CCDC bands from all seasons, assuming the sequence of bands in corr_matrix and ccdc_bands_std_devs is the same as below
    # specify columns for seasonal ccdc spectral features
    bands = ["BLUE", "GREEN", "RED", "NIR", "SWIR1", "SWIR2"]  
    spring_bands = [band + '_spring' for band in bands]
    early_summer_bands = [band + '_early_summer' for band in bands]
    late_summer_bands = [band + '_late_summer' for band in bands]
    fall_bands = [band + '_fall' for band in bands]
    # add ccdc error to each season
    spring_bands_df = dt_train[spring_bands] + ccdc_error_df
    early_summer_bands_df = dt_train[early_summer_bands] + ccdc_error_df
    mid_summer_bands_df = dt_train[bands] + ccdc_error_df
    late_summer_bands_df = dt_train[late_summer_bands] + ccdc_error_df
    fall_bands_df = dt_train[fall_bands] + ccdc_error_df

    ## generate new training features with uncertainty added
    dt_x = pd.concat([spring_bands_df,early_summer_bands_df, mid_summer_bands_df, late_summer_bands_df, fall_bands_df], axis=1) 
    dt_y = dt_train['y2']

    print(dt_x.shape)
    ## train model
    model = XGBRegressor(n_estimators=400, max_depth=12, learning_rate=0.027, colsample_bytree=0.6, subsample=0.6, tree_method='exact', max_bin=250,   n_jobs=15).fit(dt_x, dt_y)
    ## save model for map production or model projection
    model_path = model_out_path + '/' + f'ML_Round_{n}.json'
    model.save_model(model_path)
    print(model_path + ' saved!')

## Example 

### Example to add uncertainty when training model 

In [3]:
os.chdir('/uufs/chpc.utah.edu/common/home/dycelab/data/Lidar/ABoVE/model_calibration/round2/Ground_data')

#### Load data, this data include spectral features from 5 seasons, AGB,  and rmse for each band (the same for different seasons)

In [4]:
dta = pd.read_csv('Uncertainty_example_data.csv')
print(dta.shape)
dta.head(3)

(10000, 37)


,BLUE_spring,GREEN_spring,RED_spring,NIR_spring,SWIR1_spring,SWIR2_spring,BLUE_early_summer,GREEN_early_summer,RED_early_summer,NIR_early_summer,...,NIR_fall,SWIR1_fall,SWIR2_fall,BLUE_rmse,GREEN_rmse,RED_rmse,NIR_rmse,SWIR1_rmse,SWIR2_rmse,AGB
0,0.043700,0.051500,0.053250,0.1640,0.1045,0.0641,0.0370,0.04975,0.049800,0.16715,...,0.139950,0.081150,0.04655,0.0100,0.0079,0.0079,0.0070,0.0098,0.0100,160.944705
1,0.241000,0.277100,0.292900,0.3604,0.1319,0.0913,0.2123,0.25040,0.263600,0.34980,...,0.248900,0.250400,0.17570,0.0085,0.0085,0.0094,0.0086,0.0085,0.0084,0.000000
2,0.026233,0.044933,0.036767,0.2195,0.0866,0.0456,0.0257,0.04420,0.035867,0.21920,...,0.204433,0.071567,0.03510,0.0085,0.0085,0.0094,0.0086,0.0085,0.0084,23.867268


#### get band rmse for error simulation 

In [5]:
bands = ["BLUE", "GREEN", "RED", "NIR", "SWIR1", "SWIR2"]  
ccdc_rmse = dta[[band + '_rmse' for band in bands]]
ccdc_rmse.head(3)

,BLUE_rmse,GREEN_rmse,RED_rmse,NIR_rmse,SWIR1_rmse,SWIR2_rmse
0,0.0100,0.0079,0.0079,0.0070,0.0098,0.0100
1,0.0085,0.0085,0.0094,0.0086,0.0085,0.0084
2,0.0085,0.0085,0.0094,0.0086,0.0085,0.0084


#### load correlation matrix for different bands 

In [6]:
bands_corr_matrix = pd.read_csv('ccdc_bands_correlation_matrix.csv')
bands_corr_matrix

,BLUE,GREEN,RED,NIR,SWIR1,SWIR2
0,1.000000,0.959022,0.916974,0.587908,0.762235,0.795397
1,0.959022,1.000000,0.972213,0.714193,0.835188,0.864264
2,0.916974,0.972213,1.000000,0.702132,0.876536,0.905244
3,0.587908,0.714193,0.702132,1.000000,0.819595,0.690050
4,0.762235,0.835188,0.876536,0.819595,1.000000,0.957754
5,0.795397,0.864264,0.905244,0.690050,0.957754,1.000000


#### simulate 3 types of uncertainty, train model with the uncertainty included data, and save model on given path 

In [7]:
uncert_train_model(dta.drop([band + '_rmse' for band in bands],axis=1), 0, 10000, ccdc_rmse, bands_corr_matrix, y_variable_nm = 'AGB', allometry_std_dev=15,  model_out_path= '/uufs/chpc.utah.edu/common/home/dycelab/data/Lidar/ABoVE/model_calibration/round2/Ground_data')

(10000, 30)
/uufs/chpc.utah.edu/common/home/dycelab/data/Lidar/ABoVE/model_calibration/round2/Ground_data/ML_Round_0.json saved!


### Example to simulate uncertainty on predictors when projecting trained model to make AGB maps 

In [8]:
year = 2022
tile ='Bh004v006' # Bh005v004
early_summer= f'/uufs/chpc.utah.edu/common/home/dycelab2/data/ABoVE/Seasonal_bands/early_summer_bands/Early_Summer_Year_{year}_Bands_{tile}_Merged.tif'
rmse  = f'/uufs/chpc.utah.edu/common/home/dycelab2/data/ABoVE/RMSE/COG/Year_{year}_RMSE_{tile}_Merged.tif'
lc = f'/uufs/chpc.utah.edu/common/home/dycelab2/data/ABoVE/LC/Year_2022_{tile}_Merged.tif'

In [9]:
with rasterio.open(early_summer) as src:
    raster_np = src.read()/10000
bands, height, width = raster_np.shape

# Read geotiff2 (RMSE per pixel and band)
with rasterio.open(rmse) as src:
    ccdc_rmse_np = src.read()/10000  # Shape: (6, height, width)

print(raster_np.shape)
print(ccdc_rmse_np.shape)

bands_corr_matrix = pd.read_csv('ccdc_bands_correlation_matrix.csv')
bands_corr_matrix

(6, 8346, 17811)
(6, 8346, 17811)


,BLUE,GREEN,RED,NIR,SWIR1,SWIR2
0,1.000000,0.959022,0.916974,0.587908,0.762235,0.795397
1,0.959022,1.000000,0.972213,0.714193,0.835188,0.864264
2,0.916974,0.972213,1.000000,0.702132,0.876536,0.905244
3,0.587908,0.714193,0.702132,1.000000,0.819595,0.690050
4,0.762235,0.835188,0.876536,0.819595,1.000000,0.957754
5,0.795397,0.864264,0.905244,0.690050,0.957754,1.000000


In [12]:
raster_np_with_uncer = uncert_project_model(raster_np,  ccdc_rmse_np, bands_corr_matrix)

In [13]:
raster_np_with_uncer.shape

(6, 8346, 17811)